Import all packages:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import nltk
import sklearn
import io
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.datasets import load_files
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback


np.random.seed(7)

Using TensorFlow backend.


Load Train and test data:

In [2]:
from sklearn.datasets import load_files
# Read training files
reviewsTrain = load_files("C:/Users/Sadie/Desktop/project/bbc/train")
# Lets get training reviews and training labels in sepearate lists
X_train, y_train = reviewsTrain.data, reviewsTrain.target


reviewsTest = load_files("C:/Users/Sadie/Desktop/project/bbc/train")
X_test, y_test = reviewsTest.data, reviewsTest.target

Decode:

In [3]:
textTrain = []
for each in X_train:
    x = each.decode("UTF-8", errors = 'ignore')
    textTrain.append(x)
    
labelsTrain = np.array(y_train)
labelsTrain_one_hot = pd.get_dummies(labelsTrain)

textTest = []
for each in X_test:
    y = each.decode("UTF-8", errors = 'ignore')
    textTest.append(y)
    
labelsTest = np.array(y_test)
labelsTest_one_hot = pd.get_dummies(labelsTest)

tokenize, stem, remove stop words:

In [4]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
#word_tokenize accepts a string as an input, not a file.
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

for doc in range(len(textTrain)):
    clean_text_train = []
    text = (textTrain[doc])
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    for token in tokens:
        if not token in stop_words:
            clean_text_train.append((token))
    news = " ".join(clean_text_train)
    textTrain[doc] = news
    
for doc in range(len(textTest)):
    clean_text_test = []
    text = (textTest[doc])
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    for token in tokens:
        if not token in stop_words:
            clean_text_test.append((token))
    news = " ".join(clean_text_test)
    textTest[doc] = news

In [5]:
top_words = 10000
t = Tokenizer(num_words = top_words)
t.fit_on_texts(textTrain)
vocab_size = len(t.word_index) + 1
#print(vocab_size)
encoded_docs = t.texts_to_sequences(textTrain)
#print(encoded_docs)

In [8]:
max_length = 3000
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
embedding_vecor_length = 300
#print(padded_docs)

# All above steps are same as RNN in order to compare the results


CNN Model part:

In [12]:
#The number of words we want our convolutional filters to cover
filter_sizes = [1,2]
num_filters = 32 
#always default set 32 or 64
#we will have filters that slide over 1 and 2 words respectively, for a total of 2 * num_filters filters
def get_model():    
    inp = Input(shape=(max_length, ))
    x = Embedding(top_words, embedding_vecor_length, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Reshape((max_length, embedding_vecor_length, 1))(x)
#embedding layer followed by a drop out layer with rate0.1
    
    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_vecor_length), kernel_initializer='normal',activation='relu')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_vecor_length), kernel_initializer='normal',activation='relu')(x)
#convolution layers    
    maxpool_0 = MaxPool2D(pool_size=(max_length - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(max_length - filter_sizes[1] + 1, 1))(conv_1)
#max pooling layers 
    z = Concatenate(axis=1)([maxpool_0, maxpool_1])
#merge the max pooling layers

    z = Flatten()(z)
    z = Dropout(0.1)(z)
        
    outp = Dense(5, activation="sigmoid")(z)
#The last stage of a convolutional neural network (CNN) is a classifier. It is called a dense layer, which is just an 
#artificial neural network (ANN) classifier.
#And an ANN classifier needs individual features, just like any other classifier. This means it needs a feature vector.
#Therefore, you need to convert the output of the convolutional part of the CNN into a 1D feature vector, 
#to be used by the ANN part of it. This operation is called flattening. It gets the output of the convolutional layers, 
#flattens all its structure to create a single long feature vector to be used by the dense layer for the final
#classification. # https://www.quora.com/What-is-the-meaning-of-flattening-step-in-a-convolutional-neural-network

#traning model    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

model = get_model()


batch_size = 256
epochs = 3


In [14]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 3000)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 3000, 300)    3000000     input_3[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 3000, 300)    0           embedding_3[0][0]                
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 3000, 300, 1) 0           spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [21]:
import datetime
import time
start_time = datetime.datetime.now().time().strftime('%H:%M:%S')
model.fit(padded_docs, labelsTrain_one_hot, epochs=3)
end_time = datetime.datetime.now().time().strftime('%H:%M:%S')
total_time=(datetime.datetime.strptime(end_time,'%H:%M:%S') - datetime.datetime.strptime(start_time,'%H:%M:%S'))
print(total_time)

Epoch 1/3
1584/1584 [==============================] - 101s 64ms/step - loss: 0.1088 - acc: 0.9705
Epoch 2/3
1584/1584 [==============================] - 99s 63ms/step - loss: 0.0648 - acc: 0.9837
Epoch 3/3
1584/1584 [==============================] - 125s 79ms/step - loss: 0.0440 - acc: 0.9903
0:05:26


In [22]:
# converting test dataset into integers and then padding
top_words = 10000
t = Tokenizer(num_words = top_words, lower = True)
t.fit_on_texts(textTest)
vocab_size = len(t.word_index) + 1
#print(vocab_size)
encoded_docs = t.texts_to_sequences(textTest)

max_length = 3000
padded_test = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)

test result:

In [23]:
scores = model.evaluate(padded_test, labelsTest_one_hot, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 99.71%
